# **Trade Recommendation Backtesting System**

## **Introduction**

This project aims to implement a **backtesting system for trade recommendation strategies**, with a strong focus on **order execution logic** and **risk management parameters**. The system is designed to evaluate the historical performance of structured trade recommendations by simulating their execution using past market data.

Each recommendation is composed of a rich set of parameters, closely resembling real-world trading instructions, including order types (e.g., **limit**, **market**, **stop**, and **stop-limit orders**) and risk controls (e.g., **stop-loss**, **trailing-stop**, **hold**, and **take-profit** mechanisms).

The recommendations are structured with the following key fields:

**Core Fields**

- **Date:** timestamp of the recommendation
- **Name:** company name of the traded stock
- **Equity:** the stock or asset ticker under consideration
- **Scenario:** a description of the market or strategy context, the recommendation rationale

**Entry Orders:** entry orders define how and at what price a position is opened

- **Market Order:**
    - indicates immediate execution at current market price
    - since my system operates with daily prices, market orders are executed using the recommendation day's mid-price (which is defined as the average of the high and low prices)
- **Limit Order:**
    - specifies a desired entry price at which the trade should be filled
    - this can be above or below the current price, depending on the recommendation's intention (to buy on a pullback near support or breakout above resistance)
    - if the limit price is reached outside of regular trading hours (during pre-market trading or a gap at the open), the order is filled with the open price
    - until filled, the order remains active and may be modified

**Sell-Limit Orders:** sell-limit orders define profit-taking targets; they are, by definition, placed above the entry price (for long positions); in my system, such targets (exit orders) can be defined using any of the following formats:

- **Sell-Limit Order Percentage:** trigger level expressed as a percentage above the entry price
- **Sell-Limit Order Difference:** absolute currency-based offset from the entry price
- **Sell-Limit Order Exact:** a fixed stop price

**Stop or Stop-Loss Orders:** stop-loss orders (often just called "stops") are used to mitigate potential losses or to protect unrealised gains; depending on trade management, the stop level may be set below or above the entry price; in my system, stop-loss orders can be defined using the following formats:

- **Stop-Loss Order Percentage:** risk level defined as a percentage from the entry price
- **Stop-Loss Order Difference:** absolute currency-based stop from entry price
- **Stop-Loss Order Exact:** a fixed price level at which the position is exited to limit loss

**Trailing-Stop Orders:** trailing stop is a dynamic stop-loss that moves in the favourable direction only; in the case of a long position, the stop price trails behind rising prices, locking in profits while allowing further upside; in my system, trailing stops can be defined using the following formats:

- **Trailing-Stop Order Percentage:** the trailing distance is maintained as a fixed percentage below the peak price
- **Trailing-Stop Order Difference:** a pecific trailing value in absolute currency-based terms that follows price movement, maintaining a fixed distance

**Direct Commands:** these are immediate actions applied to the position, unless stated otherwise

- **Hold:** indicates a recommendation to maintain the position without any changes; during the holding period, no stop orders are activated
- **Close:** signals the immediate closing of an open position, thereby realizing current profit or loss

**The system does not:**
- support short selling or short positions, as it operates only long positions
- account for taxes, fees, or transaction costs associated with trading
- specify position sizes, as it operates with a single position at a time
- account for slippage, assuming that orders are filled at the specified prices
- consider dividends, interest, or other corporate actions
- handle multiple positions or complex order types, focusing on single-entry and single-exit scenarios
- support partial fills, assuming that orders are either fully executed or not executed at all
- open and close positions on the same day, as it operates with daily prices

**Tables and columns**

- `data_equites`: contains data tables of selected equities, named after the stock's ticker, with the following columns:

    - `date`: day of trading prices (timestamp for each trading day)
    - `open`: opening price of the equity for the trading day
    - `high`: highest price reached by the equity during the trading day
    - `low`: lowest price reached by the equity during the trading day
    - `close`: closing price of the equity for the trading day

- `data_orders`:

    - `date`: day of trade recommendation
    - `company`: name of the company for which the recommendation is made
    - `ticker`: ticker symbol of the equity
    - `scenario`: description or rationale for the trade recommendation
    - `buy_market`: indicator (1 or 0) for a market buy order
    - `buy_limit`: price for a limit buy order (buy only if price reaches this value)
    - `sell_limit_pct`: take-profit trigger as a percentage above the entry price
    - `sell_limit_dff`: take-profit trigger as an absolute value above the entry price
    - `sell_limit_xct`: take-profit trigger as an exact price
    - `sell_loss_pct`: stop-loss trigger as a percentage below the entry price
    - `sell_loss_dff`: stop-loss trigger as an absolute value below the entry price
    - `sell_loss_xct`: stop-loss trigger as an exact price
    - `sell_loss_trail_pct`: trailing stop-loss as a percentage below the highest price since entry
    - `sell_loss_trail_dff`: trailing stop-loss as an absolute value below the highest price since entry
    - `hold`: indicator to hold the position (no exit orders active)
    - `sell_market`: indicator for a market sell order (close position at current market price)

- `data_trading`:

    - all columns from `data_equites` and `data_orders`
    - `buy_price`: the price at which a buy order is executed (either market or limit)
    - `sell_price`: the price at which a sell order is executed (market, limit, or stop)
    - `sell_above`: the current active take-profit (sell-limit) price
    - `sell_below`: the current active stop-loss or trailing-stop price,
    - `trade_active`: boolean flag indicating whether a position is currently open (True) or not (False)
    - `trade_price`: the price at which the current trade was opened, and closed
    - `high_so_far`: the highest price reached since the trailing stop was ordered



In [ ]:
# Libraries –––––
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.io import to_html

from pandas.tseries.offsets import BDay
import textwrap

## **Trading Orders**

In [47]:
# Importing trading order recommendations –––––

# Reading excel
data_orders = pd.read_excel("orders.xlsx", sheet_name="recommendations", skiprows=1)

# Rename columns based on their function
data_orders = data_orders.rename(
    columns={
        "Date": "date",
        "Name": "company",
        "Equity": "ticker",
        "Scenario": "scenario",
        "Market order": "buy_market",
        "Limit order": "buy_limit",
        "Sell-Limit order\n(percentage)": "sell_limit_pct",
        "Sell-Limit order\n(difference)": "sell_limit_dff",
        "Sell-Limit order\n(exact)": "sell_limit_xct",
        "Stop-Loss order\n(percentage)": "sell_loss_pct",
        "Stop-Loss order\n(difference)": "sell_loss_dff",
        "Stop-Loss order\n(exact)": "sell_loss_xct",
        "Trailing-Stop order\n(percentage)": "sell_loss_trail_pct",
        "Trailing-Stop order\n(difference)": "sell_loss_trail_dff",
        "Hold": "hold",
        "Close": "sell_market",
    }
)

# Convert table to dictionary
# Where the keys are the equitiy names and the values are dataframes of the order recommendations
dict_orders = {
    ticker: data_orders[data_orders["ticker"] == ticker]
    .drop(columns=["ticker"])
    .reset_index(drop=True)
    for ticker in data_orders["ticker"].unique()
}

In [48]:
# Importing equity prices –––––
# Reading excel sheets as equity data frames
data_equites = pd.read_excel(
    "stocks_data.xlsx", list(dict_orders.keys()), verbose=False
)

# Modifications –
for e in data_equites.keys():
    # rename columns for consistency
    data_equites[e] = data_equites[e].rename(columns={"Date": "date",
                                                        "Open": "open",
                                                        "High": "high",
                                                        "Low": "low",
                                                        "Close": "close"})
    # Delete empty rows (if any)
    data_equites[e] = data_equites[e].dropna().reset_index(drop=True)
    # Convert date column to date format
    data_equites[e]["date"] = pd.to_datetime(data_equites[e]["date"])
    # Create mid price variable
    # It supposed to be the average of bid and ask prices,
    # but as it is used only as a reference point,
    # I calculate it with the low and high prices
    data_equites[e]["mid"] = data_equites[e][["low", "high"]].mean(axis=1)
    # Round values
    # Only for readibility purposes,
    # it should not be rounded in real life scenarios
    data_equites[e] = data_equites[e].round(2)


In [ ]:
# Backtest algorithm –––––

# Initiate dictionary for trading
data_trading = {}

# Iterate through each equity –
for e in data_equites.keys():

    # Data preparation –
    # Merge the the prices with the order recommendations
    data_trading[e] = pd.merge(data_equites[e], dict_orders[e], how="left", on="date")

    # Forward fill order variables
    data_trading[e][["buy_limit", "sell_limit_pct", "sell_limit_dff", "sell_limit_xct", "sell_loss_pct", "sell_loss_dff", "sell_loss_xct", "sell_loss_trail_pct", "sell_loss_trail_dff", "hold"]] =\
    data_trading[e][["buy_limit", "sell_limit_pct", "sell_limit_dff", "sell_limit_xct", "sell_loss_pct", "sell_loss_dff", "sell_loss_xct", "sell_loss_trail_pct", "sell_loss_trail_dff", "hold"]].infer_objects(copy=False).ffill()
    
    # Replace "-" with NA values
    data_trading[e] = data_trading[e].replace("-", None)

    # Convert to float
    data_trading[e][["buy_limit", "sell_limit_pct", "sell_limit_dff", "sell_limit_xct", "sell_loss_pct", "sell_loss_dff", "sell_loss_xct", "sell_loss_trail_pct", "sell_loss_trail_dff", "hold"]] =\
    data_trading[e][["buy_limit", "sell_limit_pct", "sell_limit_dff", "sell_limit_xct", "sell_loss_pct", "sell_loss_dff", "sell_loss_xct", "sell_loss_trail_pct", "sell_loss_trail_dff", "hold"]].astype(float)

    # Initiate columns for buy/sell prices
    data_trading[e][["buy_price", "sell_price", "sell_above", "sell_below"]] = np.nan

    # Initiate columns for track trades and prices
    data_trading[e]["trade_active"] = False
    data_trading[e]["trade_price"] = np.nan
    data_trading[e]["high_so_far"] = 0.0

    # Iterate through each observation (days) –
    for i in range(len(data_trading[e])):
        
        # BUY ORDERS –
        # Check if trade active (no)
        if not data_trading[e].loc[i, "trade_active"]:
            
            # Check for market buy order
            if data_trading[e].loc[i, "buy_market"] == 1:
                data_trading[e].loc[i:, "buy_price"] = data_trading[e].loc[i, "mid"]
                data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "mid"]
                data_trading[e].loc[i:, "trade_active"] = True
                continue
            
            # Check for limit buy order
            if data_trading[e].loc[i, "buy_limit"] > 0:
                data_trading[e].loc[i:, "buy_price"] = data_trading[e].loc[i, "buy_limit"]

            # Check if price reached the buy limit price in regular trading hours...
            if (data_trading[e].loc[i, "low"] <= data_trading[e].loc[i, "buy_price"] <= data_trading[e].loc[i, "high"]):
                data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "buy_price"]
                data_trading[e].loc[i:, "trade_active"] = True
                continue
            # ... or outside trading hours
            if (i != 0) and ((data_trading[e].loc[i-1, "close"] <= data_trading[e].loc[i, "buy_price"] <= data_trading[e].loc[i, "open"]) or 
                                (data_trading[e].loc[i-1, "close"] >= data_trading[e].loc[i, "buy_price"] >= data_trading[e].loc[i, "open"])):
                data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "open"]
                data_trading[e].loc[i:, "trade_active"] = True
                continue

        # SELL ORDERS –
        # Check if trade active (yes)
        if data_trading[e].loc[i, "trade_active"]:
            
            # Calculate sell/stop prices –

            # Sell limit variable
            # percentage
            if data_trading[e].loc[i, "sell_limit_pct"] > 0:
                data_trading[e].loc[i:, "sell_above"] = data_trading[e].loc[i, "buy_price"] * (1 + data_trading[e].loc[i, "sell_limit_pct"])
            # difference
            elif data_trading[e].loc[i, "sell_limit_dff"] > 0:
                data_trading[e].loc[i:, "sell_above"] = data_trading[e].loc[i, "buy_price"] + data_trading[e].loc[i, "sell_limit_dff"]
            # exact
            elif data_trading[e].loc[i, "sell_limit_xct"] > 0:
                data_trading[e].loc[i:, "sell_above"] = data_trading[e].loc[i, "sell_limit_xct"]
            # no sell limit
            else:
                data_trading[e].loc[i:, "sell_above"] = np.nan

            # Stop loss variable
            # percentage
            if data_trading[e].loc[i, "sell_loss_pct"] > 0:
                data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "buy_price"] * (1 - data_trading[e].loc[i, "sell_loss_pct"])
            # difference
            elif data_trading[e].loc[i, "sell_loss_dff"] > 0:
                data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "buy_price"] - data_trading[e].loc[i, "sell_loss_dff"]
            # exact
            elif data_trading[e].loc[i, "sell_loss_xct"] > 0:
                data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "sell_loss_xct"]

            # Trailing stop variable
            # percentage
            elif data_trading[e].loc[i, "sell_loss_trail_pct"] > 0:
                # calculate high so far
                if (i != 0) and data_trading[e].loc[i-1, "high_so_far"] < data_trading[e].loc[i, "high"]:
                    data_trading[e].loc[i, "high_so_far"] = data_trading[e].loc[i, "high"]
                else:
                    data_trading[e].loc[i, "high_so_far"] = data_trading[e].loc[i-1, "high_so_far"]
                # check high so far
                if (i != 0) and (data_trading[e].loc[i-1, "high_so_far"] < data_trading[e].loc[i, "high_so_far"]):
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "high"] * (1 - data_trading[e].loc[i, "sell_loss_trail_pct"])
                elif np.isnan(data_trading[e].loc[i-1, "sell_below"]):
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "high"] * (1 - data_trading[e].loc[i, "sell_loss_trail_pct"])
                else:
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i-1, "sell_below"]
            # difference
            elif data_trading[e].loc[i, "sell_loss_trail_dff"] > 0:
                # calculate high so far
                if (i != 0) and data_trading[e].loc[i-1, "high_so_far"] < data_trading[e].loc[i, "high"]:
                    data_trading[e].loc[i, "high_so_far"] = data_trading[e].loc[i, "high"]
                else:
                    data_trading[e].loc[i, "high_so_far"] = data_trading[e].loc[i-1, "high_so_far"]
                # check high so far
                if (i != 0) and (data_trading[e].loc[i-1, "high_so_far"] < data_trading[e].loc[i, "high_so_far"]):
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "high"] - data_trading[e].loc[i, "sell_loss_trail_dff"]
                elif np.isnan(data_trading[e].loc[i-1, "sell_below"]):
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i, "high"] - data_trading[e].loc[i, "sell_loss_trail_dff"]
                else:
                    data_trading[e].loc[i:, "sell_below"] = data_trading[e].loc[i-1, "sell_below"]
            # no stop loss
            else:
                data_trading[e].loc[i:, "sell_below"] = np.nan


            # Check if trade on hold -
            # (yes)
            if data_trading[e].loc[i, "hold"] == 1:
                data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "close"]
                data_trading[e].loc[i:, "trade_active"] = True
                continue
            # (no)
            else:

                # Check for market sell order
                if data_trading[e].loc[i, "sell_market"] == 1:
                    data_trading[e].loc[i, ["sell_price", "trade_price"]] = data_trading[e].loc[i, "mid"]
                    data_trading[e].loc[i+1:, ["buy_price", "sell_above", "sell_below"]] = np.nan
                    data_trading[e].loc[i:, "trade_active"] = False
                    continue

                # Check if price reached the sell limit price in regular trading hours...
                if (data_trading[e].loc[i, "low"] <= data_trading[e].loc[i, "sell_above"] <= data_trading[e].loc[i, "high"]):
                    data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "sell_above"]
                    data_trading[e].loc[i+1:, ["buy_price", "sell_above", "sell_below"]] = np.nan
                    data_trading[e].loc[i:, "trade_active"] = False
                    continue
                # ... or outside trading hours
                elif (i != 0) and ((data_trading[e].loc[i-1, "close"] <= data_trading[e].loc[i, "sell_above"] <= data_trading[e].loc[i, "open"]) or 
                                   (data_trading[e].loc[i-1, "close"] >= data_trading[e].loc[i, "sell_above"] >= data_trading[e].loc[i, "open"])):
                    data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "open"]
                    data_trading[e].loc[i+1:, ["buy_price", "sell_above", "sell_below"]] = np.nan
                    data_trading[e].loc[i:, "trade_active"] = False
                    continue


                # Check if price reached the sell limit price in regular trading hours...
                if (data_trading[e].loc[i, "low"] <= data_trading[e].loc[i, "sell_below"] <= data_trading[e].loc[i, "high"]):
                    data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "sell_below"]
                    data_trading[e].loc[i+1:, ["buy_price", "sell_above", "sell_below"]] = np.nan
                    data_trading[e].loc[i:, "trade_active"] = False
                    continue
                    
                # ... or outside trading hours
                elif (i != 0) and ((data_trading[e].loc[i-1, "close"] <= data_trading[e].loc[i, "sell_below"] <= data_trading[e].loc[i, "open"]) or 
                                   (data_trading[e].loc[i-1, "close"] >= data_trading[e].loc[i, "sell_below"] >= data_trading[e].loc[i, "open"])):
                    data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "open"]
                    data_trading[e].loc[i+1:, ["buy_price", "sell_above", "sell_below"]] = np.nan
                    data_trading[e].loc[i:, "trade_active"] = False
                    continue


                # If no sell price reached
                data_trading[e].loc[i, "trade_price"] = data_trading[e].loc[i, "close"]
                data_trading[e].loc[i:, "trade_active"] = True
                continue


In [ ]:
# Calculate metrics –––––

# Initiate dictionary for metrics
dict_metrics = {}

# Iterate through each equity –
for e in data_equites.keys():
    # Initiate dictionary for equtiy
    dict_metrics[e] = {}
    # Total return, daily return, and volatility
    dict_metrics[e]["total_return"] = data_trading[e]["trade_price"].dropna().pct_change().add(1).cumprod().sub(1).iloc[-1]
    dict_metrics[e]["daily_return"] = data_trading[e]["trade_price"].dropna().pct_change().mean()
    dict_metrics[e]["volatility"] = data_trading[e]["trade_price"].dropna().pct_change().std()

In [ ]:
# Colors –
#00BCD4
#2196F3
#00897B
#4CAF50
#808000
#00E676
#FFEB3B
#FF9800
#880E4F
#FF5252
#E040FB
#9C27B0
#311B92
#363A45
#787B86
#B2B5BE
#FFFFFF

## **Visualization**

In [53]:
# Plots –––––

# Iterate through each equity –
for e in data_equites.keys():

    # Data table for plots –
    plot_table = data_trading[e].copy()
    
    # Set range
    # Starts 5 days before the first buy order is placed
    plot_start_index = plot_table["buy_price"].first_valid_index() - 5
    # Ends 5 days after the position is closed (or on day if it is out of bundary)
    if len(plot_table) < (plot_table["trade_active"][::-1].idxmax() + 5):
        plot_end_index = plot_table["trade_active"][::-1].idxmax()
    else:
        plot_end_index = plot_table["trade_active"][::-1].idxmax() + 5
    plot_table = plot_table[plot_start_index:plot_end_index]

    # Add buy/sell order filled signals
    plot_table["buy_order_filled"] = np.where(plot_table["trade_active"].astype(int).diff() == 1,
                                              plot_table["trade_active"].astype(int).diff() * plot_table["trade_price"],
                                              np.nan)
    plot_table["sell_order_filled"] = np.where(plot_table["trade_active"].astype(int).diff() == -1,
                                               -plot_table["trade_active"].astype(int).diff() * plot_table["trade_price"],
                                               np.nan)
    
    # Separate column for trade price during hold period
    plot_table["trade_price_hold"] = np.where(plot_table["hold"] == 1,
                                              plot_table["trade_price"],
                                              np.nan)
    
    # Remove buy price after the order is filled
    plot_table.loc[plot_table["trade_active"][::1].idxmax() + 1:, "buy_price"] = np.nan

    # Order point - buy
    plot_table["buy_price_set"] = np.where(plot_table["buy_price"].fillna(0).diff() != 0,
                                           plot_table["buy_price"], np.nan)
    # Remove changes, so it appears as discontinuous
    plot_table["buy_price"] = np.where(plot_table["buy_price"].ffill().diff().shift(-1) != 0,
                                       np.nan, plot_table["buy_price"])

    # Stop loss type column for calculation
    plot_table["stop_loss_type"] = plot_table[["sell_loss_pct", "sell_loss_dff", "sell_loss_xct", "sell_loss_trail_pct", "sell_loss_trail_dff"]].notna().idxmax(axis=1)

    # Order point - sell
    plot_table["sell_above_set"] = np.where(plot_table["sell_above"].fillna(0).diff() != 0,
                                            plot_table["sell_above"], np.nan)
    # Remove changes, so it appears as discontinuous
    plot_table["sell_above"] = np.where(plot_table["sell_above"].ffill().diff().shift(-1) != 0,
                                        np.nan, plot_table["sell_above"])
    
    # Order point - stop
    plot_table["sell_below_set"] = np.where((plot_table["sell_below"].fillna(0).diff() != 0) & ((plot_table["date"].isin(dict_orders[e]["date"])) | (plot_table["date"].isin(plot_table.loc[plot_table["buy_order_filled"].notna(), "date"] + BDay(1)))),
                                            plot_table["sell_below"], np.nan)
    # Remove changes, so it appears as discontinuous
    plot_table["sell_below"] = np.where(plot_table["stop_loss_type"].isin(["sell_loss_trail_dff", "sell_loss_trail_pct"]), plot_table["sell_below"],
                                        np.where((plot_table["sell_below"].ffill().diff().shift(-1) != 0), np.nan, plot_table["sell_below"]))
    
    # Wrap long scenario text
    dict_orders[e]["scenario_wrapped"] = dict_orders[e]["scenario"].apply(
        lambda x: textwrap.fill(x, width=40).replace('\n', '<br>') if isinstance(x, str) else x)

    # Figure –
    fig = go.Figure()

    # OHLC dimmed in the backround
    fig.add_trace(go.Ohlc(x=plot_table["date"],
                          open=plot_table["open"],
                          high=plot_table["high"],
                          low=plot_table["low"],
                          close=plot_table["close"],
                          name="OHLC",
                          opacity=0.25,
                          xhoverformat="%Y-%m-%d"))

    
    # Traded price line
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["trade_price"],
                             mode="lines", name="Trade price",
                             line=dict(width=2, color="#FF9800", dash="solid"),
                             hovertemplate="%{x|%Y-%m-%d}<br><b>Traded price:</b> %{y:.2f}<extra></extra>"
                             ))
    
    # Hold price line
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["trade_price_hold"],
                             mode="lines", name="On hold",
                             line=dict(width=2, color="#252526", dash="solid"),
                             hoverinfo="skip", showlegend=False
                             ))
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["trade_price_hold"],
                             mode="lines", name="On hold",
                             line=dict(width=2, color="#FF9800", dash="dash"),
                             hovertemplate="%{x|%Y-%m-%d}<br><b>Traded price on hold:</b> %{y:.2f}<extra></extra>"
                             ))
    
    # Buy price line
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["buy_price"],
                             mode="lines", name="Buy price",
                             line=dict(width=2, color="#00E676", dash="solid"),
                             hoverinfo="skip"
                             ))
    
    # Buy price mark
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["buy_price_set"],
                             mode="markers", name="Buy price mark",
                             marker=dict(size=6, color="#00E676", symbol="circle"),
                             hovertemplate="%{x|%Y-%m-%d}<br><b>Buy price:</b> %{y:.2f}<extra></extra>"
                             ))
    
    # Buy order filled mark
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["buy_order_filled"],
                            mode="markers", name="Buy order filled",
                            marker=dict(size=12, color="#4CAF50", symbol="arrow-up"),
                            hovertemplate="%{x|%Y-%m-%d}<br><b>Buy order filled at</b> %{y:.2f}<extra></extra>"
                            ))

    
    # Sell limit price line
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["sell_above"],
                             mode="lines", name="Sell-limit price",
                             line=dict(width=2, color="#FF5252", dash="solid"),
                             hoverinfo="skip"
                             ))
    
    # Sell limit mark
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["sell_above_set"],
                             mode="markers", name="Sell-limit mark",
                             marker=dict(size=6, color="#FF5252", symbol="circle"),
                             hovertemplate="%{x|%Y-%m-%d}<br><b>Sell limit price:</b> %{y:.2f}<extra></extra>"
                             ))
    
    # Stop loss price line
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["sell_below"],
                             mode="lines", name="Stop-loss price",
                             line=dict(width=2, color="#FF5252", dash="solid"),
                             hoverinfo="skip"
                             ))
    
    # Stop loss mark
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["sell_below_set"],
                             mode="markers", name="Stop-loss mark",
                             marker=dict(size=6, color="#FF5252", symbol="circle"),
                             hovertemplate="%{x|%Y-%m-%d}<br><b>Stop loss price:</b> %{y:.2f}<extra></extra>"
                             ))
    
    # Sell order filled mark
    fig.add_trace(go.Scatter(x=plot_table["date"], y=plot_table["sell_order_filled"],
                            mode="markers", name="Sell order filled",
                            marker=dict(size=12, color="#DC2626", symbol="arrow-down"),
                            hovertemplate="%{x|%Y-%m-%d}<br><b>Sell order filled at</b> %{y:.2f}<extra></extra>"
                            ))
    
    # Recommendation mark
    fig.add_trace(go.Scatter(x=dict_orders[e]["date"], y=np.repeat([plot_table[["low", "sell_below"]].min().min() * 0.95], len(dict_orders[e]["date"])),
                        mode="markers", name="Recommendation",
                        marker=dict(size=9, color="#2196F3", symbol="circle-open-dot"),
                        text=dict_orders[e]["scenario_wrapped"],
                        hovertemplate="%{x|%Y-%m-%d} <b>Recommendation:</b><br><i>%{text}</i><extra></extra>"
                        ))

    
    # Update -
    # Hide non-trading days
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"], pattern="day of week")])

    # Update layout
    fig.update_layout(

        # Main title
        title="<b>Trade Recommendation Backtest System Visualised</b>",
        title_font=dict(family="Montserrat", size=22, color="#FFFFFF"), #Montserrat-Black

        # X-Axis
        xaxis=dict(
            title="Date",
            titlefont=dict(family="Montserrat", size=14, color="#FFFFFF"), #Montserrat-Bold
            tickfont=dict(family="Montserrat", size=12, color="#FFFFFF"),
            linecolor='#787B86', gridcolor='#363A45', zerolinecolor='#787B86',
            showline=True, showgrid=True, zeroline=False),

        # Y-Axis
        yaxis=dict(
            title="Price",
            titlefont=dict(family="Montserrat", size=14, color="#FFFFFF"), #Montserrat-Bold
            tickfont=dict(family="Montserrat", size=12, color="#FFFFFF"),
            linecolor='#787B86', gridcolor='#363A45', zerolinecolor='#787B86',
            showline=True, showgrid=True, zeroline=False),

        # Legend
        legend=dict(
            y=1, x=1,
            title=dict(
                text="<b>Legend</b>",
                font=dict(family="Montserrat", size=14, color="#FFFFFF")), #Montserrat-Bold
            font=dict(family="Montserrat", size=12, color="#FFFFFF")
        ),

        # General layout
        font=dict(family="Montserrat", size=12, color="#FFFFFF"),
        
        plot_bgcolor="#252526",
        paper_bgcolor="#252526",
        
        autosize=False,
        height=500, width=1000,
        margin=dict(l=40, r=40, t=80, b=40),
        xaxis_rangeslider_visible=False
    )

    # Subtitle
    fig.add_annotation(
        text=f"Company: {plot_table.loc[plot_table['company'].first_valid_index(), 'company']} | Ticker: {e} | Position: Long",
        xref="paper", yref="paper",
        x=-0.025, y=1.09,
        showarrow=False,
        align="left",
        font=dict(family="Montserrat", size=16, color="#B2B5BE")
    )

    # Annotation
    fig.add_annotation(
        text="<b>Summary</b>",
        xref="paper", yref="paper",
        x=1.11, y=0.2,
        showarrow=False,
        align="left",
        font=dict(family="Montserrat", size=14, color="#FFFFFF") #Montserrat-Bold
    )
    plot_annotation_text = "<br>".join([
        " ",
        f"• Total Return: <b>{round(dict_metrics[e]['total_return'] * 100, 2)}%</b>",
        f"• Daily Return: <b>{round(dict_metrics[e]['daily_return'] * 100, 2)}%</b>",
        f"• Volatility: <b>{round(dict_metrics[e]['volatility'] * 100, 2)}%</b>"
    ])
    fig.add_annotation(
        text=plot_annotation_text,
        xref="paper", yref="paper",
        x=1.17, y=0.06,
        showarrow=False,
        align="left",
        font=dict(family="Montserrat", size=12, color="#FFFFFF")
    )
    
    # Show figure –
    fig.show()

    # Save the figure as an HTML file –
    with open(f"figures/TRBS_{e}.html", "w") as f:
        f.write(to_html(fig, include_plotlyjs="cdn", full_html=True))

    # Inject Montserrat font link into the HTML head
    with open(f"figures/TRBS_{e}.html", "r+") as f:
        content = f.read()
        insert_pos = content.find("</head>")
        if insert_pos != -1:
            content = content[:insert_pos] + '<link href="https://fonts.googleapis.com/css?family=Montserrat:400,700,900&display=swap" rel="stylesheet">\n' + content[insert_pos:]
            f.seek(0)
            f.write(content)
            f.truncate()
